In [29]:
import numpy as np
import tensorflow as tf

import sys
import os

os.environ['KERAS_BACKEND']='tensorflow'

import re

In [41]:
from keras.layers import Input, Dense, Layer
from keras.models import Model

from keras import backend as K

In [97]:
import tensorflow as tf

#define a variable to hold normal random values 
normal_rv = tf.Variable( tf.truncated_normal([1,10],stddev = 1))

#initialize the variable
init_op = tf.global_variables_initializer()

#run the graph
with tf.Session() as sess:
    sess.run(init_op) #execute init_op
    #print the random values that we sample
    print (sess.run(normal_rv))

[[ 0.02038496 -0.5463682  -0.7210144   0.31411332  1.3295283   0.44473293
   0.0554226   0.7371137   0.9403496  -0.88727677]]


In [40]:
class SparseMax(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(SparseMax, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [49]:
first_input = Input(shape=(10,), dtype='float32')
last_output = Dense(3, activation='softmax')(first_input)

model = Model(first_input, last_output)

In [44]:
random_input = np.random.rand(1,10)

In [45]:
random_input.shape

(1, 10)

In [46]:
y = [0]

In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 33        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [51]:
def softmax(x, axis=-1):
    """Softmax activation function.
    # Arguments
        x: Input tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D. '
                         'Received input: %s' % x)

In [98]:
get = sparsemax(normal_rv)

In [100]:
with tf.Session() as sess:
    sess.run(init_op) #execute init_op
    #print the random values that we sample
#     sess.run(get)
    print(sess.run(normal_rv))
    print (sess.run(get))
    

[[ 1.2181993   0.7580004  -1.0405164  -0.9009907   0.45127302 -1.116122
  -1.4705615   1.6638165  -0.4929908   0.31552693]]
[[0.2771914 0.        0.        0.        0.        0.        0.
  0.7228086 0.        0.       ]]


In [84]:
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn

In [85]:
def sparsemax(logits, name=None):
    with ops.name_scope(name, "sparsemax", [logits]) as name:
        logits = ops.convert_to_tensor(logits, name="logits")
        obs = array_ops.shape(logits)[0]
        dims = array_ops.shape(logits)[1]

        z = logits - math_ops.reduce_mean(logits, axis=1)[:, array_ops.newaxis]

        # sort z
        z_sorted, _ = nn.top_k(z, k=dims)

        # calculate k(z)
        z_cumsum = math_ops.cumsum(z_sorted, axis=1)
        k = math_ops.range(
            1, math_ops.cast(dims, logits.dtype) + 1, dtype=logits.dtype)
        z_check = 1 + k * z_sorted > z_cumsum
        # because the z_check vector is always [1,1,...1,0,0,...0] finding the
        # (index + 1) of the last `1` is the same as just summing the number of 1.
        k_z = math_ops.reduce_sum(math_ops.cast(z_check, dtypes.int32), axis=1)

        # calculate tau(z)
        indices = array_ops.stack([math_ops.range(0, obs), k_z - 1], axis=1)
        tau_sum = array_ops.gather_nd(z_cumsum, indices)
        tau_z = (tau_sum - 1) / math_ops.cast(k_z, logits.dtype)

        # calculate p
        return math_ops.maximum(
            math_ops.cast(0, logits.dtype), z - tau_z[:, array_ops.newaxis])